In [14]:
from pathlib import Path
import cv2
import numpy as np
from openvino.runtime import Core

# 初始化 OpenVINO Core
core = Core()

# 加载电动车和头盔检测模型
model_path = Path("bicycle_weights/best_openvino_model/best.xml")
compiled_model = core.compile_model(model=model_path, device_name="CPU")

model_path_helmet = Path("runs/detect/train7/weights/best_openvino_model/best.xml")
compiled_model_helmet = core.compile_model(model=model_path_helmet, device_name="CPU")

# 测试图片集
test_images = ["test_image_1.jpg", "test_image_2.png"]
final_results = []

# 确保所有输入图像尺寸一致，适配网络输入要求
target_size = (512, 512)  # 假定模型需要512x512尺寸的输入
imagenet_labels = {  0: "car",
  1: "threewheel",
  2: "bus",
  3: "truck",
  4: "motorbike",
  5: "van"}
def postprocess(outs):
    score = np.max(outs)
    id = np.argmax(outs)
    return score, id, imagenet_labels[id]

# 处理每张图片
for image_path in test_images:
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)  # 调整图像尺寸
    image_height, image_width, _ = image.shape

    # 预处理图像
    input_blob = np.expand_dims(image.transpose(2, 0, 1), 0)  # 转换为模型所需的格式
    # 执行推理
    results = compiled_model.infer_new_request({0: input_blob})
    break
    # 后处理
outs = results[compiled_model.outputs[0]]
print(np.max(outs))
print(np.argmax(outs))
#     detections = results.get_tensor('detection_output').data  # 替换为实际输出层名
#     for detection in detections:
#         # 筛选出电动车的检测结果
#         if int(detection[1]) == 4:  # 假设类别ID为4代表电动车
#             x1, y1, x2, y2 = map(int, detection[3:7] * [image_width, image_height, image_width, image_height])
#             # 裁剪图像以检测头盔
#             cropped_image = image[y1:y2, x1:x2]
#             cropped_image = cv2.resize(cropped_image, target_size)  # 调整裁剪图像尺寸
#             cropped_blob = np.expand_dims(cropped_image.transpose(2, 0, 1), 0)
# 
#             # 头盔检测推理
#             helmet_results = compiled_model_helmet.infer_new_request({0: cropped_blob})
#             helmet_detections = helmet_results.get_tensor('detection_output').data
# 
#             # 处理头盔检测结果
#             for helmet_detection in helmet_detections:
#                 if int(helmet_detection[1]) == 0:  # 假设类别ID为0代表头盔
#                     hx1, hy1, hx2, hy2 = map(int, helmet_detection[3:7] * [x2 - x1, y2 - y1, x2 - x1, y2 - y1])
#                     cv2.rectangle(cropped_image, (hx1, hy1), (hx2, hy2), (0, 255, 0), 2)  # 绿色框表示头盔
# 
#             # 将处理后的区域写回原始图像
#             image[y1:y2, x1:x2] = cropped_image
# 
#             # 保存结果
#             final_results.append({
#                 'scooter_bbox': (x1, y1, x2, y2),
#                 'helmet_detection': helmet_detections
#             })
# 
#     # 保存标注后的图像
#     annotated_image_path = f'annotated_{Path(image_path).stem}.jpg'
#     cv2.imwrite(annotated_image_path, image)
# 
# # 输出处理结果
# for result in final_results:
#     print(result)


507.4549
3903
